In [1]:
import numpy as np 
import pandas as pd 
from collections import defaultdict, Counter
import wget
from Bio import SeqIO
from Bio import AlignIO
from Bio.Alphabet import generic_protein
from Bio.Align.Applications import ClustalwCommandline
from Bio.Align.Applications import MuscleCommandline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from pathlib import Path
import os



# do imports
import os, io, random
import string
import numpy as np

from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

# import panel as pn
# import panel.widgets as pnw
# pn.extension()

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot
from IPython.display import display, HTML
from bokeh.io import output_file, show, save, curdoc, output_notebook
from bokeh.io import export_png
import matplotlib.pyplot as plt


BOOK_ROOT = os.path.dirname(os.path.realpath('__file__'))
DATA_PATH = Path(Path(BOOK_ROOT).resolve().parent, "data")
TOOLS_PATH = Path(Path(BOOK_ROOT).resolve().parent, "tools")
PLOTS_PATH = Path(Path(BOOK_ROOT).resolve().parent, "plots")
RESULTS_PATH = Path(Path(BOOK_ROOT).resolve().parent, "results")

In [2]:
print(BOOK_ROOT)

/home/aneesh/Projects/covid_bh_ml/master/orf1ab/notebooks


# Download and install muscle aligner

In [3]:
# !cd $TOOLS_PATH

# filename = wget.download("http://www.drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86linux64.tar.gz")

# !tar xvzf $filename
# !rm muscle3.8.31_i86linux64.tar.gz 
# !cd $BOOK_ROOT

In [4]:
class FastaMeta:
    '''
    object to hold fasta file and its corresponding meta file paths
    '''
    def __init__(self, fasta, meta):
        '''
        fasta: .fa file name
        meta: .csv file name
        '''
        self.data_path = 'https://raw.githubusercontent.com/covid19-bh-machine-learning/master/master/data/'
        # self.git_data_path = Path(Path(__file__).resolve().parent.parent.parent, "orf1ab-pyCode")
        self.fasta = os.path.join(self.data_path, fasta)
        self.meta = os.path.join(self.data_path, meta)

class DataProcessing(FastaMeta):
    def __init__(self, fasta, meta):
        super().__init__(fasta, meta)

    def get_amino_df(self):
        '''
        k = kmer length
        Generates all possible offsets of amino acid sequence and
        returns a pandas dataframe merged with given metadata
        meta_format ; csv tsv etc
        '''
        meta_df = pd.read_csv(self.meta, header=0)
        seq_seq = defaultdict(list)
        filename = self.fasta.split('/')[-1]
        if not os.path.exists(filename):
            filename = wget.download(self.fasta)
        seq_list = list(SeqIO.parse(filename, 'fasta', alphabet=generic_protein))
        for s in seq_list:
            seq_seq['seq'].append(str(s.seq))
        meta_df['seq'] = seq_seq['seq']
        return meta_df

In [5]:
def get_data(orf1):  
    # read for data folder and out put 
    df = orf1.get_amino_df()
    print(f"shape WITH duplicates: {df.shape}")

    # remove duplicates
    df.drop_duplicates(subset='Accession', keep=False, inplace=True)
    print(f"shape WITHOUT duplicates: {df.shape}")
    df['Collection_Date'] = pd.to_datetime(df['Collection_Date'], errors='coerce').dt.strftime('%Y-%m-%d')
    df['Release_Date'] = pd.to_datetime(df['Release_Date'], errors='coerce').dt.strftime('%Y-%m-%d')
    df['Length'] = df['Length'].apply(str)
    return df

def filter_column(df, column_name, min_count):
    '''
    df: dataframe
    column_name: column to filter
    min_count: minimum count required to be included
    '''
    counts = Counter(df[column_name])
    filtered = [key for key in counts if counts[key] >= min_count]
    print(filtered)
    df = df[df[column_name].isin(filtered)]
    return df[df[column_name].notna()]

def map_classes(df, column_name):
    #labels
    lbl_enc = LabelEncoder()
    y = lbl_enc.fit_transform(df[column_name].values)

    # map labels to numercial values
    #map labels to numerical value
    labels = list(lbl_enc.inverse_transform(y))
    return dict(zip(y, labels)), y

def get_test_data(df, column_name):
    
    class_dict, y = map_classes(df, column_name)
    #train test split
    xtrain, xvalid, ytrain, yvalid = train_test_split(df['seq'].values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)
    return xvalid, yvalid
   

In [6]:

def get_task_dataframe(tasks):
    orf1 = DataProcessing('coronavirus_orf1ab.fasta', 'coronavirus_orf1ab_meta.csv')
    data = get_data(orf1)
    df_species = None
    df_host = None
    df_geo = None
    df_source = None
    tasks_dict = dict(zip(tasks, [df_source, df_host, df_geo, df_species]))
    for column_name in tasks_dict:
        df = filter_column(data, column_name, 20)
        a1, _ = get_test_data(df, column_name)
        print(len(df), len(a1))
        tasks_dict[column_name] = df[df['seq'].isin(a1)]
    return tasks_dict



In [7]:
# select sequences from the test data for futher analysis
    
def get_test_data_from_fasta(task):
    '''
    Isolate the test data from fasta recrods
    '''
    tasks_dict[task].drop_duplicates(subset="seq", inplace=True)
    tasksdf = tasks_dict[task].sample(20, random_state=42)
    with open(f'{DATA_PATH}/orf1ab_{task}_test.fasta', 'w') as outFile:
        record_ids = tasksdf['Accession'].values
        for record in SeqIO.parse(f'{DATA_PATH}/coronavirus_orf1ab.fasta', 'fasta'):
            if record.id in record_ids:
                SeqIO.write(record, outFile, 'fasta')

In [8]:
# remove duplicate sequences

def remove_duplicate_fasta_ids(fasta_name):
    with open(f'{fasta_name.split(".")[0]}_no_duplicates.fasta', 'w') as outFile:
        record_ids = set()
        for record in SeqIO.parse(fasta_name, 'fasta'):
            if record.id not in record_ids:
                record_ids.add(record.id)
                SeqIO.write(record, outFile, 'fasta')

In [9]:
#perform multiple sequence alignment
def clustal_align(fasta_name):
    cline = ClustalwCommandline(f"{TOOLS_PATH}/clustalw2", infile=fasta_name)
    cline()
    align = AlignIO.read(f"{DATA_PATH}/{fasta_name.split('.')[0]}.aln", "clustal")
    return align

In [10]:
#MUSCLE

def muscle_align(fasta_file):
    out_file = f"{fasta_file.split('.')[0]}_aligned.fasta"
    muscle_exe = f"{TOOLS_PATH}/muscle3.8.31_i86linux64"
    muscle_cline = MuscleCommandline(muscle_exe, input=fasta_file, out=out_file)
    stdout, stderr = muscle_cline()
    return AlignIO.read(out_file,'fasta')

def maaft_align(fasta_file):
    from Bio.Align.Applications import MafftCommandline
    out_file = f"{fasta_file.split('.')[0]}_aligned.fasta"
    mafft_exe = f"{TOOLS_PATH}/mafft-linux64/mafft.bat"
    mafft_cline = MafftCommandline(mafft_exe, input=fasta_file)
    stdout, stderr = mafft_cline()
    with open(out_file, "w") as handle:
        handle.write(stdout)
    return AlignIO.read(out_file,'fasta')

In [11]:
def make_seq(length=40):    
    return ''.join([random.choice(['A','C','T','G']) for i in range(length)])

def mutate_seq(seq):
    """mutate a sequence randomly"""
    seq = list(seq)
    pos = np.random.randint(1,len(seq),6)    
    for i in pos:
        seq[i] = random.choice(['A','C','T','G'])
    return ''.join(seq)

def get_colors(seqs):
    """make colors for bases in sequence"""
    text = [i for s in list(seqs) for i in s]
    clrs =  {'G':'orange', 'P': 'orange', 'S':'orange', 'T':'orange', 'A':'orange',
            'H':'red', 'K':'red', 'R':'red', 'E':'red', 'D':'red',
            'F':'blue', 'W':'blue', 'Y':'blue',
            'I':'green', 'L':'green', 'M':'green', 'V':'green', 'C':'green',
            '-':'grey', 'X':'grey',
            'N':'magenta', 'Q':'magenta'}
#     G, P, S, T	Orange
# H, K, R	Red
# F, W, Y	Blue
# I, L, M, V	Green
    colors = [clrs[i] for i in text]
    return colors


In [12]:
def get_top_kmer_list(task, ntop):
    species_result_path = os.path.join(RESULTS_PATH, task)
    files = os.listdir(species_result_path)
    kmer_dict = defaultdict(list)
    for file in files:
        if 'feature' in file:
            df = pd.read_csv(os.path.join(species_result_path, file))
            df.sort_values("weight", ascending=False, inplace=True)
            features = df['feature'].values.tolist()
            
            for f in features:
                if 'BIAS' not in f:
                    if len(kmer_dict[file.split('.')[0]]) <= ntop:
                        kmer_dict[file.split('.')[0]].append(f)
                    else:
                        break
    return kmer_dict



def make_kmer_fasta(record_dict, fasta_name):
    with open(fasta_name, 'w') as handle:
        for entry in record_dict:
            for i, rec in enumerate(record_dict[entry]):
                handle.write(f">{entry}_{i}\n{rec}\n")
            


In [13]:

def print_fasta_records(fasta_name, num_records=5):
    for record in SeqIO.parse(fasta_name, 'fasta')[:num_records]:
        print(record.seq)
    


def combine_fasta(filenames, out_name):
    with open(out_name, 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
                    


In [14]:
# aligner ref https://dmnfarrell.github.io/bioinformatics/bokeh-sequence-aligner

def view_alignment(aln, window, fontsize="9pt", plot_width=800):
    """Bokeh sequence alignment view"""
    output_notebook()
    #make sequence and id lists from the aln object
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]    
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)    
    N = len(seqs[0])
    S = len(seqs)    
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    #creates a 2D grid of coords from the 1D arrays
    xx, yy = np.meshgrid(x, y)
    #flattens the arrays
    gx = xx.ravel()
    gy = yy.flatten()
    #use recty for rect coords with an offset
    recty = gy+.5
    h= 1/S
    #now we can create the ColumnDataSource with all the arrays
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*15+50
    x_range = Range1d(0,N+1, bounds='auto')
#     if N>2000:
#         viewlen=2000
#     else:
#         viewlen=N
    #view_range is for the close up view
    view_range = window
    tools="xpan, wheel_zoom, reset, save"

    #entire sequence view (no text, with zoom)
    p = figure(title=None, plot_width= plot_width, plot_height=50,
               x_range=view_range, y_range=(0,S), tools=tools,
               min_border=0, toolbar_location='below')
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                 line_color=None, fill_alpha=0.6)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False  

    #sequence text view with ability to scroll along x axis
    p1 = figure(title=None, plot_width=plot_width, plot_height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)          
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                text_font="monospace",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)

    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0
    

    p = gridplot([[p],[p1]], toolbar_location='below') #, sizing_mode='stretch_both'
#     p.sizing_mode = 'scale_both'
    export_png(p, filename=f'{PLOTS_PATH}/plot.png')
#     plt.savefig("plot.png")
#     curdoc().add_root(p)
#     output_file('plot.html', mode='inline')
#     output_file("bars.html")
#     show(p)
#     display(HTML('plot.html'))
#     return p


In [15]:
def view_alignment_split(seqs, ids, split, file_name, fontsize="9pt", plot_width=800):
    """Bokeh sequence alignment view"""
    output_notebook()
    #make sequence and id lists from the aln object
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)    
    N = len(seqs[0])
    S = len(seqs)    
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    #creates a 2D grid of coords from the 1D arrays
    xx, yy = np.meshgrid(x, y)
    #flattens the arrays
    gx = xx.ravel()
    gy = yy.flatten()
    #use recty for rect coords with an offset
    recty = gy+.5
    h= 1/S
    #now we can create the ColumnDataSource with all the arrays
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*15+50
    x_range = Range1d(0,N+1, bounds='auto')

    #view_range is for the close up view
    view_range = (0,N)#max(N*i, 1), max(N+1, (N*i)+1
    tools="xpan, wheel_zoom, reset, save"

    #entire sequence view (no text, with zoom)
    p = figure(title=None, plot_width= plot_width, plot_height=50,
               x_range=view_range, y_range=(0,S), tools=tools,
               min_border=0, toolbar_location='below')
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                 line_color=None, fill_alpha=0.6)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False  

    #sequence text view with ability to scroll along x axis
    p1 = figure(title=None, plot_width=plot_width, plot_height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)          
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                text_font="monospace",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)
#     p1.extra_x_ranges = {"pos": Range1d((N*i)+1, max(N, N*(i+1)+1))}
    
    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0
#     p1.x_range = Range1d(N*split, max(N, N*(split+1)))
    

    p = gridplot([[p],[p1]], toolbar_location='below') #, sizing_mode='stretch_both'
#     p.sizing_mode = 'scale_both'
    export_png(p, filename=f'{PLOTS_PATH}/{file_name}_plot_{split}.png')
#     plt.savefig("plot.png")
#     curdoc().add_root(p)
#     output_file('plot.html', mode='inline')
#     output_file("bars.html")
#     show(p)
#     display(HTML('plot.html'))
#     return p


In [16]:
def split_and_plot_alignment(aln, task, split_size = 1000, max_len = 9000, plot_width=10000):
    seqs = [[rec.seq[i:i+split_size] for rec in (aln)] for i in range(0, max_len, split_size)]
    ids = [[rec.id for rec in (aln) ] for seq in seqs]  
    for i in range(len(seqs)):
        view_alignment_split(seqs[i], ids[i], i, f'_{task}_kmer', plot_width=plot_width)

In [22]:
def align_and_plot_kmer(task):

    kmer_dict = get_top_kmer_list(task, 1)  
    del kmer_dict['orf1_Species_1_mer_lr_feature']
    kmer_fasta_path = f'{DATA_PATH}/kmer_{task}_lr_fasta.fasta'
    make_kmer_fasta(kmer_dict, kmer_fasta_path)


    filenames = [kmer_fasta_path, f'{DATA_PATH}/orf1ab_Species_test.fasta']
    out_fasta = f'{DATA_PATH}/kmer_{task}_lr_fasta_merged.fasta'
    combine_fasta(filenames, out_fasta)
    return out_fasta



In [23]:

tasks = ['Species', 'Host', 'Isolation_Source', 'Geo_Location']
tasks_dict = get_task_dataframe(tasks)

for task in tasks:
    get_test_data_from_fasta(task)


############IMPORTANT - SET THE TASK NAME FIRST################


task = 'Species'

############IMPORTANT - SET THE TASK NAME FIRST################
out_fasta = align_and_plot_kmer(task)

aln = muscle_align(out_fasta)
#     aln = maaft_align(out_fasta)


shape WITH duplicates: (3046, 10)
shape WITHOUT duplicates: (2384, 10)
['Betacoronavirus 1', 'Coronavirus HKU15', 'Human coronavirus 229E', 'Middle East respiratory syndrome-related coronavirus', 'Alphacoronavirus 1', 'Avian coronavirus', 'Human coronavirus HKU1', 'Human coronavirus NL63', 'Severe acute respiratory syndrome-related coronavirus', 'Porcine epidemic diarrhea virus', 'Alphacoronavirus sp.']
2190 219
[nan, 'Sus scrofa', 'Camelus', 'Homo sapiens', 'Chiroptera', 'Gallus gallus', 'Scotophilus kuhlii', 'Camelus dromedarius', 'Felis catus', 'Rhinolophus sinicus', 'Mus musculus']
1855 186
[nan, 'feces', 'abdominal cavity', 'oronasopharynx', 'lung, oronasopharynx']
767 77
['USA', 'Hong Kong', 'China: Hong Kong', 'China', 'Kenya', 'Saudi Arabia', nan, 'Colombia', 'South Korea', 'Viet Nam', 'China: Beijing', 'United Arab Emirates', 'USA: Illinois', 'USA: Minnesota', 'USA: Ohio', 'USA: Nashville, TN', 'USA: Denver, CO', 'USA: Tennessee']
1801 181


In [25]:
split_and_plot_alignment(aln, task)

Loading BokehJS ...

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=60737): Max retries exceeded with url: /session/bd3da4b31f1e1a0b3777fd2f95423562/window/current/maximize (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f131393fef0>: Failed to establish a new connection: [Errno 111] Connection refused',))